<a href="https://colab.research.google.com/github/Siralde/PlantVillageHomeWork/blob/main/PlantVillage_Tarea.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds

In [3]:
(train_data, test_data), dataset_info = tfds.load(
    'plant_village',
    split=['train[:60%]', 'train[60%:]'],
    with_info=True,
    as_supervised=True
)

In [4]:
def preprocess(image, label):
    image = tf.image.resize(image, (150, 150))
    image = tf.cast(image, tf.float32) / 255.0  # Normalización
    return image, label

batch_size = 32

train_batches = train_data.shuffle(10000).batch(batch_size).map(preprocess).prefetch(tf.data.experimental.AUTOTUNE)
test_batches = test_data.batch(batch_size).map(preprocess)

In [5]:
def data_augmentation(image, label):
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_flip_up_down(image)
    image = tf.image.random_brightness(image, max_delta=0.3)
    image = tf.image.random_contrast(image, lower=0.7, upper=1.3)
    return image, label

augmented_train_batches = train_batches.map(data_augmentation)

In [6]:
num_classes = dataset_info.features['label'].num_classes

In [7]:
print(f'Número de clases: {num_classes}')

Número de clases: 38


In [ ]:
model = tf.keras.models.Sequential([
    # Primera capa convolucional
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2,2),
    # Segunda capa convolucional
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # Tercera capa convolucional
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # Aplanamiento
    tf.keras.layers.Flatten(),
    # Capa densa
    tf.keras.layers.Dense(512, activation='relu'),
    # Capa de salida
    tf.keras.layers.Dense(num_classes, activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

history = model.fit(augmented_train_batches, validation_data=test_batches, epochs=10)

Epoch 1/10
 593/1019 [================>.............] - ETA: 13:56 - loss: 1.7166 - accuracy: 0.5200